# Gene Ontology (GO) Molecular Function

Author: Zachary Flamholz  
Date: 06-2018  
Database: https://wormbase.org/#012-34-5  
Data: http://geneontology.org/gene-associations/gene_association.wb.gz  
Companion files: http://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Invertebrates/Caenorhabditis_elegans.gene_info.gz, http://snapshot.geneontology.org/ontology/go-basic.obo

## import libraries

In [1]:
import numpy as np
import pandas as pd
import sys, datetime
import goenrich
import scipy.stats as stat

# Versions of Modules in Use

In [2]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget 

Software versions
Python 2.7.15 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 5.7.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.3
pandas 0.23.0
clustergrammer_widget The 'clustergrammer_widget' distribution was not found and is required by the application
Tue Jul 10 12:08:50 2018 EDT

## read in data

In [3]:
df = pd.read_csv('input/wb.gaf', sep='~', skiprows=14, header=None)

In [4]:
df.head()

,0
0,WB\tWBGene00000001\taap-1\t\tGO:0005942\tGO_RE...
1,WB\tWBGene00000001\taap-1\t\tGO:0005942\tPMID:...
2,WB\tWBGene00000001\taap-1\t\tGO:0008286\tPMID:...
3,WB\tWBGene00000001\taap-1\t\tGO:0008286\tPMID:...
4,WB\tWBGene00000001\taap-1\t\tGO:0008286\tPMID:...


In [5]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
matrix = np.chararray((df.shape[0], 17), itemsize=150, unicode=True)

for i, row in enumerate(df.itertuples()):
    
    progressPercent = ((i+1)/len(df.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
    sys.stdout.flush()
    
    lst = row[1].split('\t')

    matrix[i, :] = lst

df_clean = pd.DataFrame(data=matrix)

In [6]:
df_clean.shape

In [7]:
columns = ["DB", "DB gene ID", "Gene symbol", "Qualifier", "GO ID", "Reference", "Evidence code", "Evidence from", "GO class", "attribute", "Locus tag", "gene/protein", "tax id", "date", "Assigned by", "additional information", "empty"]

In [8]:
df_clean.columns = columns

In [9]:
df_clean.head(10)

In [10]:
df_function = df_clean[df_clean["GO class"] == 'F'].copy()

In [11]:
# remove any annotation assigned by electronic matching, and with the NOT qualifier which is used to specify a gene is not associated with a term
df_function = df_function[df_function["Evidence code"] != 'IEA']
df_function = df_function[df_function["Qualifier"] != 'NOT']

In [12]:
df_function = df_function[['Gene symbol', 'GO ID']]
df_function.reset_index(inplace=True)
df_function = df_function.drop(columns = 'index')

In [13]:
df_function.shape

In [14]:
df_function.head(10)

## load GO digraph

In [15]:
digraph = goenrich.obo.ontology('input/go-basic.ob')

## only keep GO IDs with depth =>4, replace others with NaN and then remove those rows


In [16]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 

lst = []

for i,index in enumerate(df_function.index):
    
    progressPercent = ((i+1)/len(df_function.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df_function.index)))
    sys.stdout.flush()
    
    term = df_function.loc[index, 'GO ID']
    if term in digraph.node:
        if digraph.node[term]['depth'] >= 4:
            lst.append(term)
        else:
            lst.append(np.nan)
    else:
        lst.append(np.nan)
           
df_function['GO ID'] = lst

In [17]:
df_function.shape

In [18]:
df_function.dropna(inplace=True)

In [19]:
df_function.shape

## term propagation-propergate child gene term relationships to parent terms

In [20]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
lst1 = []
lst2 = []

for i,index in enumerate(df_function.index):
    
#     progressPercent = ((i+1)/len(df.index))*100

#     sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
#     sys.stdout.flush()
    
#     term = df.loc[index, 'GO ID']
#     for parent in digraph.predecessors(term):
#         if parent in digraph.node:
#             if digraph.node[parent]['depth'] >= 4:
#                 lst1.append(df.loc[index, 'DB Object Symbol'])
#                 lst2.append(parent)
#                 print(term, parent)
    term = df_function.loc[index, 'GO ID']
    for parent in digraph.successors(term):
        if parent in digraph.node:
            if digraph.node[parent]['depth'] >= 4:
                lst1.append(df_function.loc[index, 'Gene symbol'])
                lst2.append(parent)
#                 print(term, parent)


temp = pd.DataFrame()
temp['Gene symbol'] = lst1
temp['GO ID']  = lst2
df_function = pd.concat([df_function, temp])
df_function.reset_index(inplace=True)
df_function = df_function.drop(columns = 'index')

In [21]:
df_function.shape

In [22]:
df_function.head()

## map GO ID to descriptive name

In [23]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
lst = []

for i,index in enumerate(df_function.index):
    
    progressPercent = ((i+1)/len(df_function.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df_function.index)))
    sys.stdout.flush()
    
    lst.append(str(digraph.node[df_function.loc[index, 'GO ID']]['name'])+' '+ '('+str(df_function.loc[index, 'GO ID'])+')')
    
df_function['GO ID'] = lst

In [24]:
df_function.shape

In [25]:
df_function.drop_duplicates(inplace=True)

In [26]:
df_function.shape

In [27]:
df_function.head()

In [28]:
df_function.reset_index(inplace=True)

In [29]:
df_function.head()

In [30]:
df_function = df_function.drop(columns = 'index')

In [31]:
df_function.shape

In [32]:
len(df_function['GO ID'].unique())

 ## build the protein-coding gene list from companion file

In [33]:
cElegans_geneInfo = pd.read_csv("input/Caenorhabditis_elegans.gene_info-1", sep="\t")

In [34]:
cElegans_geneInfo.head()

In [35]:
cElegans_proteinCoding = cElegans_geneInfo[cElegans_geneInfo["type_of_gene"] == "protein-coding"]

In [36]:
cElegans_proteinCoding.shape

## only keep gene symbolys that are in the C elegans table

In [37]:
df_function_protein = df_function.loc[df_function["Gene symbol"].isin(cElegans_proteinCoding["Symbol"].unique())]

In [38]:
df_function_protein.shape

In [39]:
filename = 'function_gene_ontology_wormbase_greater4_%s.gmt'% str(datetime.date.today())[0:7].replace('-', '_')
file = open(filename,'w+') 
terms = df_function_protein["GO ID"].unique()
for i,term in enumerate(terms):
    
    progressPercent = ((i+1)/len(terms))*100
    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(terms)))
    sys.stdout.flush()
            
    df_byTerm = df_function_protein.loc[df_function_protein["GO ID"] == term]
    
    if df_byTerm.shape[0] > 4:
        # split splice variant names. some genes have format X.#.# where the second decimal describes a TX ID
        split_splice = lambda x: (x.split('.')[0] + x.split('.')[1]) if len(x.split('.')) > 2 else x
        df_byTerm["Gene symbol"] = df_byTerm["Gene symbol"].apply(split_splice)
        
        
        if len(df_byTerm["Gene symbol"].unique()) > 4:
            file.write("%s\t" % term)
            file.write("\t")
            genes = df_byTerm["Gene symbol"].unique()
    
            for gene in genes:
                file.write("%s\t" % gene)
            file.write("\n")
        
file.close()

## get stats for gmt

In [40]:
def getTermStats(loaded_gmt, geneInfoTable, num_samplings, num_genes_in_sampling) :
    # get the relevant terms and set a dictionary for them
    term_set = loaded_gmt.iloc[:,0].apply(lambda x: x.split("\t")[0])
    term_genes_dict = {term_set[i]: loaded_gmt.iloc[i,0].split("\t")[2:] for i in range(0, loaded_gmt.shape[0])}
    term_rank_dict = {k: [] for k in term_set}
    term_stats_dict = {k: [] for k in term_set}
    
    # get the total number of genes in the organism
    n_genes = len(geneInfoTable["Symbol"].unique())
    
    # number of genes per sampling, number of times to sample
    genes_in_sampleing = num_genes_in_sampling
    num_samplings = num_samplings
    
    for i,x in enumerate(range(0,num_samplings)):
        
        progressPercent = ((i+1)/len(range(0,num_samplings)))*100
        
        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(range(0,num_samplings))))
        sys.stdout.flush()
            
        
        randomSet = set(geneInfoTable["Symbol"].sample(genes_in_sampleing))
        
        # initialize an array to hold the pvalue for each term
        pvals = np.array([])
        for key in term_rank_dict.keys():
            
            termSet = set(term_genes_dict[key])
            # calculate p value using fisher exact test
            # implemented using the formula found in the GeneOverap bioconductor package for R
            pval_term = stat.fisher_exact([[n_genes - len(termSet.union(randomSet)), len(randomSet.difference(termSet))], [len(termSet.difference(randomSet)), len(termSet.intersection(randomSet))]])[1]
            pvals = np.append(pvals, [pval_term])
        
        # sort the pvals and add the rank to the term_rank dict
        sorted_pvals = pvals.argsort()
        
        # find the rank for each term by sorting the pvals array and getting the index of the key position in the
        # sorted list. Need to +1 because the index begins with 0
        
        for j,key in enumerate(term_rank_dict.keys()):
            term_rank_dict[key].append(np.where(sorted_pvals == j)[0][0] + 1)
            
    for key in term_rank_dict:
        term_stats_dict[key] = [np.mean(term_rank_dict[key]), np.std(term_rank_dict[key])]
            
    
    
    return term_stats_dict

In [41]:
function_GO_gmt = pd.read_csv("function_gene_ontology_wormbase_greater4_2018_06.gmt", sep="~", header=None)
type(function_GO_gmt)

pandas.core.frame.DataFrame

In [42]:
iterations_10 = getTermStats(function_GO_gmt, cElegans_proteinCoding, num_samplings=10, num_genes_in_sampling=300)

In [43]:
iterations_200 = getTermStats(function_GO_gmt, cElegans_proteinCoding, num_samplings=200, num_genes_in_sampling=300)

In [44]:
df_200_iterations = pd.DataFrame.from_dict(iterations_200, orient='index', columns = ['mean', 'sd'])

In [45]:
df_200_iterations.to_csv('function_gene_ontology_wormbase_greater4_2018_06_stats.tsv', sep='\t', header=False)